### functionality

### required directory structure

### imports and inputs

In [61]:
import numpy as np
import pandas as pd
import seaborn as sb
import math
import statistics as stat
import os

from packages import miscellaneous as mc
from packages import timeconvert as tc
from packages import plotstyle as ps

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
sb.set_theme(style="white",font="serif")

graph = 'z'

date = '8_9_2021 (early morning)'

timeRange = ["00","06"] # Start and end hour in 24-hour time: ["HH","HH"]. For title only.

timeToScale = False # If true, tick labels are daily. Else, tick labels occur only on observation days.

numOutliers = 2 # Number of outliers you'd like to exclude. If not needed, set to 0.

# Code is general to antennas, but plot arrangements are specific, so structure of lists below is fixed.
refants = ['DA59','DV11','DA49','DV12','PM01','DV07'] # Must be three pairs [A,A,B,B,C,C], where A,A are central.
pairLoc = ['Center','Southwest','East'] # Must be three corresponding locations [A,B,C].

directory = '/Users/colemeyer/Documents/ALMA Internship/Observational Data/' # Directory of "Observational Data", assuming specified structure

### general initializations

In [62]:
weaants = ['Meteo129','Meteo130','Meteo131','Meteo201','Meteo309','Meteo410','MeteoCentral']

directory = directory+date
genDir = directory+'/asdm/'

numObs = mc.numObs(genDir)

if graph == 'x':
    index = 0
elif graph == 'y':
    index = 1
else:
    index = 2
    
# Create new "offsetplots" directory within "Observational Data" subdirectory
if not os.path.isdir(directory+'/phaseplots/'):
    os.mkdir(directory+'/phaseplots/')
    
# Prepare x-values           NOTE: WILL NEED TO BE MODIFIED BASED ON MONTH
tempArr = pd.read_csv(directory+'/offsetdata/'+refants[0]+'.csv')
timeList = [tc.UnixtoFormattedTime(int(ele)) for ele in tempArr.loc[:,'time']]
x = []
for time in timeList:
    if time[1] == "7":
        x.append(int(time[3:5]))
    else: x.append(int(time[3:5])+31)
offset = x[0]-1
for i in range(len(x)): x[i] = x[i] - offset
x = np.array(x)
minTimeLabel = str(int(timeList[0][:2]))+'-'+str(int(timeList[0][3:5]))
maxTimeLabel = str(int(timeList[-1][:2]))+'-'+str(int(timeList[-1][3:5]))
maxTimeIndex = x[-1]

# Prepare time labels
timeSeries = ['',minTimeLabel]
while timeSeries[-1] != maxTimeLabel:
    if timeSeries[-1] != "7-31":
        temp = timeSeries[-1].split('-')
        timeSeries.append(temp[0]+'-'+str(int(temp[1])+1))
    else: timeSeries.append('8-1')

# magnitude w/ mean-based normalization

### specific initializations

In [63]:
magProbs = np.empty((np.shape(refants)[0],numObs),dtype='U25')
mag = np.empty((np.shape(refants)[0],numObs,6),dtype='float')
if timeToScale: 
    xMag = x
    xticks = range(maxTimeIndex+1)
else:
    xMag = range(1,numObs+1)
    xticks = range(numObs+1)
    
    # Prepare time labels
    timeSeries = []
    temp = pd.read_csv(directory+'/offsetdata/'+refants[0]+'.csv')[['time']]
    for time in range(temp.shape[0]):
        timeSeries.append(tc.UnixtoFormattedTime(temp.loc[time]))
    
    timeSeries.append("")

########## magnitude-specific pos data

# Data extraction from offsetdata directory
for ant in range(np.shape(refants)[0]):
    arr = pd.read_csv(directory+'/offsetdata/'+refants[ant]+'.csv')
    arr = arr.loc[:,'poserrX':'poserr()Z']
    
    # Mean-based normalization
    for coord in range(3):
        offset = np.average(arr.iloc[:,coord].fillna(0))
        arr.iloc[:,coord] = arr.iloc[:,coord]-offset
    
    # Eliminating outliers
    tempList = arr.iloc[:,index]
    tempList = tempList.fillna(0).abs().sort_values(ascending=False)
    if numOutliers != 0:
        for row in range(numOutliers):
            arr.loc[tempList.index[row],:] = 0
            
    for obs in range(numObs):
        # If there's a problem in the data, prep it for the next loop
        if arr.iloc[obs].isnull().values.any():
            L = np.where(magProbs[ant,:] == "")[0][0]
            magProbs[ant,L] = obs
            mag[ant,obs,:] = np.nan
        # Else, transfer the data
        else: mag[ant,obs,:] = arr.iloc[obs]
            
# If there's a problem in the data, take the data from the previous observation (or the next, if the previous is
# not available) and update magProbs, which will allow us to circle the problem areas in our plot.
for ant in range(np.shape(refants)[0]):
    if magProbs[ant,0] != "":
        for prob in range(np.where(magProbs[ant,:] == "")[0][0]):
            k = 1
            if int(magProbs[ant,prob]) == 0:
                while np.isnan(np.sum(mag[ant,int(magProbs[ant,prob])+k,:])):
                    k += 1
                    
                mag[ant,int(magProbs[ant,prob]),:] = mag[ant,int(magProbs[ant,prob])+k,:]
            else:
                while np.isnan(np.sum(mag[ant,int(magProbs[ant,prob])-k,:])):
                    k += 1
                    
                mag[ant,int(magProbs[ant,prob]),:] = mag[ant,int(magProbs[ant,prob])-k,:]
                
                
magPhaseProbs = np.empty((np.shape(refants)[0],numObs),dtype='U25')
magPhase = np.empty((np.shape(refants)[0],numObs),dtype='float')
                
########## magnitude-specific phase data

# Data extraction from offsetdata directory
for ant in range(np.shape(refants)[0]):
    arr = pd.read_csv(directory+'/offsetdata/'+refants[ant]+'.csv')
    arr = arr.loc[:,'xPhaseAvg']
    
    # Mean-based normalization
    offset = np.average(arr)
    arr = arr-offset
    
    # Eliminating outliers
    tempList = arr
    tempList = tempList.fillna(0).abs().sort_values(ascending=False)
    if numOutliers != 0:
        for row in range(numOutliers):
            arr.loc[tempList.index[row]] = 0
    
    for obs in range(numObs):
        # If there's a problem in the data, prep it for the next loop
        if np.isnan(arr.iloc[obs]):
            L = np.where(magPhaseProbs[ant,:] == "")[0][0]
            magPhaseProbs[ant,L] = obs
            magPhase[ant,obs] = np.nan
        # Else, transfer the data
        else: magPhase[ant,obs] = arr.iloc[obs]
            
# If there's a problem in the data, take the data from the previous observation (or the next, if the previous is
# not available) and update magPhaseProbs, which will allow us to circle the problem areas in our plot.
for ant in range(np.shape(refants)[0]):
    if magPhaseProbs[ant,0] != "":
        for prob in range(np.where(magPhaseProbs[ant,:] == "")[0][0]):
            if int(magPhaseProbs[ant,prob]) == 0: magPhase[ant,int(magPhaseProbs[ant,prob])] = magPhase[ant,int(magPhaseProbs[ant,prob])+1]
            else: magPhase[ant,int(magPhaseProbs[ant,prob])] = magPhase[ant,int(magPhaseProbs[ant,prob])-1]

### combined plot initializations

In [64]:
magProbsAvg = np.empty((int(np.shape(refants)[0]/2),numObs),dtype='U25')
magAvg = np.zeros((int(np.shape(refants)[0]/2),numObs,6),dtype='float')

for ant in range(int(np.shape(refants)[0]/2)):
    magAvg[ant,:] = (mag[ant*2,:] + mag[ant*2+1,:]) / 2
        
for ant in range(np.shape(refants)[0]):
    pair = ant // 2
    if magProbs[ant,0] != "":
        for prob in range(np.where(magProbs[ant,:] == "")[0][0]):
            L = np.where(magProbsAvg[pair,:] == "")[0][0]
            magProbsAvg[pair,L] = magProbs[ant,prob]
for pair in range(int(np.shape(refants)[0]/2)):
    if magProbsAvg[pair,0] != "":
        for prob in range(0):
            if int(magProbsAvg[pair,prob]) == 0: magAvg[pair,int(magProbsAvg[pair,prob]),:] = magAvg[pair,int(magProbsAvg[pair,prob])+1,:]
            else: magAvg[pair,int(magProbsAvg[pair,prob]),:] = magAvg[pair,int(magProbsAvg[pair,prob])-1,:]

                
for pair in range(int(np.shape(refants)[0]/2)):
    for coord in range(3):
        offset = np.average(magAvg[pair,:,coord])
        magAvg[pair,:,coord] = magAvg[pair,:,coord] - offset
        
magPhaseProbsAvg = np.empty((int(np.shape(refants)[0]/2),numObs),dtype='U25')
magPhaseAvg = np.zeros((int(np.shape(refants)[0]/2),numObs),dtype='float')

for ant in range(int(np.shape(refants)[0]/2)):
    magPhaseAvg[ant,:] = (magPhase[ant*2,:] + magPhase[ant*2+1,:]) / 2
        
for ant in range(np.shape(refants)[0]):
    pair = ant // 2
    if magPhaseProbs[ant,0] != "":
        for prob in range(np.where(magPhaseProbs[ant,:] == "")[0][0]):
            L = np.where(magPhaseProbsAvg[pair,:] == "")[0][0]
            magPhaseProbsAvg[pair,L] = magPhaseProbs[ant,prob]
for pair in range(int(np.shape(refants)[0]/2)):
    if magPhaseProbsAvg[pair,0] != "":
        for prob in range(0):
            if int(magPhaseProbsAvg[pair,prob]) == 0: magPhaseAvg[pair,int(magPhaseProbsAvg[pair,prob]),:] = magPhaseAvg[pair,int(magPhaseProbsAvg[pair,prob])+1]
            else: magPhaseAvg[pair,int(magPhaseProbsAvg[pair,prob]),:] = magPhaseAvg[pair,int(magPhaseProbsAvg[pair,prob])-1]

                
for pair in range(int(np.shape(refants)[0]/2)):
    offset = np.average(magPhaseAvg[pair,:])
    magPhaseAvg[pair,:] = magPhaseAvg[pair,:] - offset

In [65]:
posPoints = []
negPoints = []
for i in range(numObs):
    if magAvg[0,i,index] > 0: posPoints.append(magAvg[0,i,index])     
avg1 = np.average(np.array(posPoints,dtype='float'))

if os.listdir(genDir)[0][:3] == "uid": uid = os.listdir(genDir)[0]
else: uid = os.listdir(genDir)[1]

avg2 = np.empty((3),dtype='float')
for i in range(3):
    posPoints = []
    negPoints = []
    for j in range(numObs):
        if magPhaseAvg[i,j] > 0: posPoints.append(magPhaseAvg[i,j])
    avg2[i] = np.average(np.array(posPoints,dtype='float'))

scaleFactor = avg1 / avg2

for i in range(3): magPhaseAvg[i,:] = magPhaseAvg[i,:] * scaleFactor[i]

/Users/colemeyer/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/Users/colemeyer/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


### extrema

In [66]:
extrema = np.zeros((int(np.shape(refants)[0]/2),2),dtype='float')
for pair in range(int(np.shape(refants)[0]/2)):
    extrema[pair,0], extrema[pair,1] = max(np.max(magAvg[pair,:,index]+magAvg[pair,:,index+3]),np.max(magPhaseAvg[pair,:])), min(np.min(magAvg[pair,:,index]-magAvg[pair,:,index+3]),np.min(magPhaseAvg[pair,:]))
    
temp = np.max([abs(np.max(extrema[:,0])), abs(np.min(extrema[:,1]))])
ymaxMag, yminMag = temp * 1.25, -(temp * 1.25)

## plot 

In [67]:
fig, ax = plt.subplots(1,3,figsize=(15,5),sharey=True)

for i in range(3):
    temp = pd.DataFrame(np.column_stack((xMag,magAvg[i,:,index],magAvg[i,:,index+3])),columns=['x', 'y','err'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='r')
    ax[i].errorbar(temp.loc[:,'x'],temp.loc[:,'y'],temp.loc[:,'err'],color='r',capsize=3)
    
    temp = pd.DataFrame(np.column_stack((xMag,-magPhaseAvg[i,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='b')

handles = []
for i in range(3):
    handles.append([Line2D([0],[0],marker='o',color='r',label='Offset Avg',linewidth=0),
                  Line2D([0],[0],marker='o',color='b',label='Phase Avg',linewidth=0)])

names = [pairLoc[0]+' Pair Avg.',pairLoc[1]+' Pair Avg.',pairLoc[2]+' Pair Avg.']
for i in range(3):
    ps.plotNonDescript(ax[i],title=graph+'-Position Offset vs. Phase Avg., '+names[i]+',\n'
                       +str(numObs)+'-Obs. Period, '+timeRange[0]+':00 - '+timeRange[1]+':00, Mean-Based Normalization',ymax=ymaxMag,ymin=yminMag,titlesize=11,xlabel='Observation Date',
                       ylabel=graph+'-Position Offset (m) vs. Phase Avg',xticks=xticks,xticklabels=timeSeries,yticks=True,handles=handles[i])

plt.tight_layout()

#plt.show()
if timeToScale: plt.savefig(directory+'/phaseplots/mag'+graph.capitalize()+'SCALED.png', bbox_inches='tight')
else: plt.savefig(directory+'/phaseplots/mag'+graph.capitalize()+'.png', bbox_inches='tight')
plt.close()

# delta w/ mean-based normalization

### specific initializations

In [68]:
deltaProbs = np.empty((np.shape(refants)[0],numObs-1),dtype='U25')
delta = np.empty((np.shape(refants)[0],numObs-1,6),dtype='float')
if timeToScale: 
    xDelta = x[1:]
    xticks = np.arange(maxTimeIndex+1)+0.5
else:
    xDelta = range(1,numObs)
    xticks = np.array(range(numObs+1),dtype='float')-0.5
    
    # Prepare time labels
    timeSeries = []
    temp = pd.read_csv(directory+'/offsetdata/'+refants[0]+'.csv')[['time']]
    for time in range(temp.shape[0]):
        timeSeries.append(tc.UnixtoFormattedTime(temp.loc[time]))
    
    timeSeries.append("")
    
########## delta-specific pos data

# Data extraction from offsetdata directory
for ant in range(np.shape(refants)[0]):
    arr = pd.read_csv(directory+'/offsetdata/'+refants[ant]+'.csv')
    arr = arr.loc[:,'poserrX':'poserr()Z']
    
    # Mean-based normalization
    for coord in range(3):
        offset = np.average(arr.iloc[:,coord].fillna(0))
        arr.iloc[:,coord] = arr.iloc[:,coord]-offset
    
    # Eliminating outliers
    tempList = arr.iloc[:,index]
    tempList = tempList.fillna(0).abs().sort_values(ascending=False)
    if numOutliers != 0:
        for row in range(numOutliers):
            arr.loc[tempList.index[row],:] = 0

    for obs in range(numObs-1):
        # If there's a problem in the data, prep it for the next loop
        if arr.iloc[obs+1].isnull().values.any() or arr.iloc[obs].isnull().values.any():
            L = np.where(deltaProbs[ant,:] == "")[0][0]
            deltaProbs[ant,L] = obs
            delta[ant,obs,:] = np.nan
        # Else, transfer the data
        else: 
            delta[ant,obs,:3] = arr.iloc[obs+1,:3]-arr.iloc[obs,:3]
            if arr.iat[obs+1,index+3] > arr.iat[obs,index+3]:
                delta[ant,obs,3:] = arr.iloc[obs+1,3:]
            else: delta[ant,obs,3:] = arr.iloc[obs,3:]
            
# If there's a problem in the data, take the data from the previous observation (or the next, if the previous is
# not available) and update deltaProbs, which will allow us to circle the problem areas in our plot.
for ant in range(np.shape(refants)[0]):
    if deltaProbs[ant,0] != "":
        for prob in range(np.where(deltaProbs[ant,:] == "")[0][0]):
            k = 1
            if int(deltaProbs[ant,prob]) == 0:
                while np.isnan(np.sum(delta[ant,int(deltaProbs[ant,prob])+k,:])):
                    k += 1
                    
                delta[ant,int(deltaProbs[ant,prob]),:] = delta[ant,int(deltaProbs[ant,prob])+k,:]
            else:
                while np.isnan(np.sum(delta[ant,int(deltaProbs[ant,prob])-k,:])):
                    k += 1
                    
                delta[ant,int(deltaProbs[ant,prob]),:] = delta[ant,int(deltaProbs[ant,prob])-k,:]
                
                
deltaPhaseProbs = np.empty((np.shape(refants)[0],numObs-1),dtype='U25')
deltaPhase = np.empty((np.shape(refants)[0],numObs-1),dtype='float')

########## deltaPhase-specific pos data

# Data extraction from offsetdata directory
for ant in range(np.shape(refants)[0]):
    arr = pd.read_csv(directory+'/offsetdata/'+refants[ant]+'.csv')
    arr = arr.loc[:,'xPhaseAvg']

    offset = np.average(deltaPhase[ant,:])
    deltaPhase[ant,:] = deltaPhase[ant,:]-offset
    
    # Eliminating outliers
    tempList = arr.iloc[:]
    tempList = tempList.fillna(0).abs().sort_values(ascending=False)
    if numOutliers != 0:
        for row in range(numOutliers):
            arr.loc[tempList.index[row]] = 0
    
    for obs in range(numObs-1):
        # If there's a problem in the data, prep it for the next loop
        if np.isnan(arr.iloc[obs+1]) or np.isnan(arr.iloc[obs]):
            L = np.where(deltaPhaseProbs[ant,:] == "")[0][0]
            deltaPhaseProbs[ant,L] = obs
            deltaPhase[ant,obs] = np.nan
        # Else, transfer the data
        else: deltaPhase[ant,obs] = arr.iloc[obs+1]-arr.iloc[obs]

# If there's a problem in the data, take the data from the previous observation (or the next, if the previous is
# not available) and update deltaPhaseProbs, which will allow us to circle the problem areas in our plot.
for ant in range(np.shape(refants)[0]):
    if deltaPhaseProbs[ant,0] != "":
        for prob in range(np.where(deltaPhaseProbs[ant,:] == "")[0][0]):
            if int(deltaPhaseProbs[ant,prob]) == 0: deltaPhase[ant,int(deltaPhaseProbs[ant,prob])] = deltaPhase[ant,int(deltaPhaseProbs[ant,prob])+1]
            else: deltaPhase[ant,int(deltaPhaseProbs[ant,prob])] = deltaPhase[ant,int(deltaPhaseProbs[ant,prob])-1]

### combined plot initializations

In [69]:
deltaProbsAvg = np.empty((int(np.shape(refants)[0]/2),numObs-1),dtype='U25')
deltaAvg = np.zeros((int(np.shape(refants)[0]/2),numObs-1,6),dtype='float')

for ant in range(int(np.shape(refants)[0]/2)):
    deltaAvg[ant,:] = (delta[ant*2,:] + delta[ant*2+1,:]) / 2
        
for ant in range(np.shape(refants)[0]):
    pair = ant // 2
    if deltaProbs[ant,0] != "":
        for prob in range(np.where(deltaProbs[ant,:] == "")[0][0]):
            L = np.where(deltaProbsAvg[pair,:] == "")[0][0]
            deltaProbsAvg[pair,L] = deltaProbs[ant,prob]
for pair in range(int(np.shape(refants)[0]/2)):
    if deltaProbsAvg[pair,0] != "":
        for prob in range(0):
            if int(deltaProbsAvg[pair,prob]) == 0: deltaAvg[pair,int(deltaProbsAvg[pair,prob]),:] = deltaAvg[pair,int(deltaProbsAvg[pair,prob])+1,:]
            else: deltaAvg[pair,int(deltaProbsAvg[pair,prob]),:] = deltaAvg[pair,int(deltaProbsAvg[pair,prob])-1,:]

                
for pair in range(int(np.shape(refants)[0]/2)):
    for coord in range(3):
        offset = np.average(deltaAvg[pair,:,coord])
        deltaAvg[pair,:,coord] = deltaAvg[pair,:,coord] - offset
        
        
deltaPhaseProbsAvg = np.empty((int(np.shape(refants)[0]/2),numObs-1),dtype='U25')
deltaPhaseAvg = np.zeros((int(np.shape(refants)[0]/2),numObs-1),dtype='float')

for ant in range(int(np.shape(refants)[0]/2)):
    deltaPhaseAvg[ant,:] = (deltaPhase[ant*2,:] + deltaPhase[ant*2+1,:]) / 2
        
for ant in range(np.shape(refants)[0]):
    pair = ant // 2
    if deltaPhaseProbs[ant,0] != "":
        for prob in range(np.where(deltaPhaseProbs[ant,:] == "")[0][0]):
            L = np.where(deltaPhaseProbsAvg[pair,:] == "")[0][0]
            deltaPhaseProbsAvg[pair,L] = deltaPhaseProbs[ant,prob]
for pair in range(int(np.shape(refants)[0]/2)):
    if deltaPhaseProbsAvg[pair,0] != "":
        for prob in range(0):
            if int(deltaPhaseProbsAvg[pair,prob]) == 0: deltaPhaseAvg[pair,int(deltaPhaseProbsAvg[pair,prob])] = deltaPhaseAvg[pair,int(deltaPhaseProbsAvg[pair,prob])+1]
            else: deltaPhaseAvg[pair,int(deltaPhaseProbsAvg[pair,prob])] = deltaPhaseAvg[pair,int(deltaPhaseProbsAvg[pair,prob])-1]

                
for pair in range(int(np.shape(refants)[0]/2)):
    for coord in range(3):
        offset = np.average(deltaPhaseAvg[pair,:])
        deltaPhaseAvg[pair,:] = deltaPhaseAvg[pair,:] - offset

In [70]:
posPoints = []
negPoints = []
for i in range(numObs-1):
    if deltaAvg[0,i,index] > 0: posPoints.append(deltaAvg[0,i,index])     
avg1 = np.average(np.array(posPoints,dtype='float'))

if os.listdir(genDir)[0][:3] == "uid": uid = os.listdir(genDir)[0]
else: uid = os.listdir(genDir)[1]

avg2 = np.empty((3),dtype='float')
for i in range(3):
    posPoints = []
    negPoints = []
    for j in range(numObs-1):
        if deltaPhaseAvg[i,j] > 0: posPoints.append(deltaPhaseAvg[i,j])
    avg2[i] = np.average(np.array(posPoints,dtype='float'))

scaleFactor = avg1 / avg2

for i in range(3): deltaPhaseAvg[i,:] = deltaPhaseAvg[i,:] * scaleFactor[i]

### extrema

In [71]:
extrema = np.zeros((int(np.shape(refants)[0]/2),2),dtype='float')
for pair in range(int(np.shape(refants)[0]/2)):
    extrema[pair,0], extrema[pair,1] = max(np.max(deltaAvg[pair,:,index]+deltaAvg[pair,:,index+3]),np.max(deltaPhaseAvg)), min(np.min(delta[ant,:,index]-delta[ant,:,index+3]),np.min(deltaPhaseAvg))
    
temp = np.max([abs(np.max(extrema[:,0])), abs(np.min(extrema[:,1]))])
ymaxDelta, yminDelta = temp * 1.25, -(temp * 1.25)

## plot

In [72]:
fig, ax = plt.subplots(1,3,figsize=(15,5),sharey=True)

for i in range(3):
    temp = pd.DataFrame(np.column_stack((xDelta,deltaAvg[i,:,index],deltaAvg[i,:,index+3])),columns=['x', 'y','err'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='r')
    ax[i].errorbar(temp.loc[:,'x'],temp.loc[:,'y'],temp.loc[:,'err'],color='r',capsize=3)
    
    temp = pd.DataFrame(np.column_stack((xDelta,-deltaPhaseAvg[i,:])),columns=['x', 'y'])
    sb.lineplot(ax=ax[i], x='x', y='y', data=temp, color='b')
    
'''
colors = ['r','b','g']
for i in range(3):
    temp = pd.DataFrame(np.column_stack((xDelta,deltaAvg[i,:,index],deltaAvg[i,:,index+3])),columns=['x', 'y','err'])
    sb.lineplot(ax=ax[3], x='x', y='y', data=temp, color=colors[i])
    ax[3].errorbar(temp.loc[:,'x'],temp.loc[:,'y'],temp.loc[:,'err'],color=colors[i],capsize=3)
    
for ant in range(len(refants)):
    if deltaProbs[ant,0] != "":
        for row in range(np.where(deltaProbs[ant,:] == "")[0][0]):
            ec = colors[ant % 2]
            ax[ant//2].scatter(float(deltaProbs[ant,row])+1,float(delta[ant,int(deltaProbs[ant,row]),index]),s=500,c='none',edgecolor=ec,linewidth=1)
            
    if deltaProbsAvg[ant//2,0] != "":
        for row in range(np.where(deltaProbsAvg[ant//2,:] == "")[0][0]):
            ec = colors[ant // 2]
            ax[3].scatter(float(deltaProbsAvg[ant//2,row])+1,float(deltaAvg[ant//2,int(deltaProbsAvg[ant//2,row]),index]),s=500,c='none',edgecolor=ec,linewidth=1)
'''

handles = []
for i in range(3):
    handles.append([Line2D([0],[0],marker='o',color='r',label='Offset Avg',linewidth=0),
                  Line2D([0],[0],marker='o',color='b',label='Phase Avg',linewidth=0)])

names = [pairLoc[0]+' Pair, ',pairLoc[1]+' Pair, ',pairLoc[2]+' Pair, ','Combined, ']
for i in range(3):
    ps.plotNonDescript(ax[i],title='test',ymax=ymaxMag,ymin=yminMag,titlesize=11,xlabel='SAMPLE LABEL',
                       ylabel='SAMPLE LABEL',xticks=xticks,xticklabels=timeSeries,yticks=True,handles=handles[i])

handles = []
for i in range(3):
    handles.append([Line2D([0],[0],marker='o',color='r',label='ΔOffset Avg',linewidth=0),
          Line2D([0],[0],marker='o',color='b',label='ΔPhase Avg',linewidth=0)])
            
names = [pairLoc[0]+' Pair Avg.',pairLoc[1]+' Pair Avg.',pairLoc[2]+' Pair Avg.']
for i in range(3):
    ps.plotNonDescript(ax[i],title=graph+'-Position ΔOffset vs. ΔPhase Avg., '+names[i]+',\n'
                       +str(numObs)+'-Obs. Period, '+timeRange[0]+':00 - '+timeRange[1]+':00, Mean-Based Normalization',ymax=ymaxDelta,ymin=yminDelta,titlesize=11,xlabel='Observation Date',
                       ylabel=graph+'-Position ΔOffset (m) vs. ΔPhase Avg',xticks=xticks,xticklabels=timeSeries,yticks=True,handles=handles[i])

plt.tight_layout()

#plt.show()
if timeToScale: plt.savefig(directory+'/phaseplots/delta'+graph.capitalize()+'SCALED.png', bbox_inches='tight')
else: plt.savefig(directory+'/phaseplots/delta'+graph.capitalize()+'.png', bbox_inches='tight')
plt.close()

print("done")

done
